# NaCl ioonsus

Kui küsida koolilapselt, mis on naatriumkloriidis kloori ja naatriumi laengud, siis ta teab kohe, et need on $-1$ ja $1$. Keerukatesse füüsikaliste mõõtmismeetoditesse tungimata, saaks eelnevale väitele varju heita näiteks kristallograafiast tuntud nähtuste abil.

Ioonilised ained esinevad tahkes olekus kristallidena, mida iseloomustab kõrge korrapära. Struktureeritus võimaldab ioonilist kristalli kirjeldada atomistilike mõõtmetega ühikraku abil, mille kordamisel igas suunas saame makroskoopilise reaalse kristalli.

Sõltuvalt ioonide suurustest ning laengutest moodustuvad erinevad kristallvõred nõnda, et elektrostaatilised interaktsioonid vastavas süsteemis oleks tugevaimaid. Kui eeldada täielikku ioonset sidet, siis peaks samasuguste suuruse- ning laengusuhetega katioonid/anioonid moodustama eranditult identseid ioonvõresid. 

Näiteks naatriumkloriidi tüüpi võres moodustab üks ioon kuubilise tihepakendi ning vastasioon täitab kõiki oktagonaalseid tühimikke. Saavuatakse (6,6)-koordinatsioon. Sellise võre moodustavad paljud kristallid nt NaCl, KBr, AgCl, MgO, CaC2, KCN, etc. Selles võres on tüüpiliselt katiooni/aniooni raadiuste suhe vahemikus 0.414-0.732 ning ioonide laengud suhtuvad 1:1.

Seevastu nikkelarseniid ja nikkelsulfiid, millel on samauti 1:1 laengute suhe ning samas vahemikus katiooni/aniooni raadiuste suhe, moodustavad hoopis deformeerunud heksagonaalse kristallvõre. Vastasioonid täidavad samuti oktagonaalseid tühimikke ning seega on koordinatsiooniarv identne NaCl võrele.

Miks siis NiAs ja NiS moodustavad Coulomb'i seaduse järgi energeetiliselt ebastabiilsema kristallvõre? Parim loogiline põhjendus on kovalentse sideme esinemine. Erinevalt ioonilisest sidemest on kovalentne side suunaline, mis tuleneb orbitaalide ruumilisest paigutusest. See energia, mida NiAs ja NiS kaotavad elektrostaatikast, võidavad nad orbitaalide kattumisega tulenevast energiaefektist.

Kui kahe erineva elektronegatiivsusega elemendi aatomi vahel on kovalentne side, siis on teada, et elektronpilv nihkub pigem elektronegatiivsema aatomi suunas. Mis suunas ja ulatuses võiks aga toimuda laenguülekanne katiooni ja aniooni vahel? Vaatleme seda naatriumkloriidi näitel.

In [ ]:
#IMPORT MODULES
from ase import Atoms
from ase.build import bulk
from ase.constraints import FixAtoms
from ase.io import write, read
from ase.optimize import QuasiNewton
from ase.units import Bohr
from ase.visualize import view
from gpaw import GPAW, FermiDirac, PoissonSolver, Mixer, setup_paths, restart
from gpaw.lcao.tools import remove_pbc, get_lcao_hamiltonian
from gpaw.xc.functional import XCFunctional
#from gpaw.xc.hybridg import HybridXC
from gpaw.test import equal
from gpaw.xc.tools import vxc
from gpaw.wavefunctions.pw import PW
from math import sqrt
import os
import numpy as np
import matplotlib.pyplot as plt
from ase.io.cube import read_cube_data
from mendeleev import Na
from mendeleev import Cl
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib.mlab import bivariate_normal

In [ ]:
#MOLECULE VIEWER CODE
from IPython.display import HTML
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

Laenguülekanne võiks olla proportsionaalne elementide elektronegatiivsuste vahega ning pöördvõrdeline nende kõvaduse summaga. Paulingi skaalas on kahe aatomi elektronegatiivsuste vahe energiavõit, mille need elemendid saavad heteronukleaarse sideme moodustamisel (vs vastavad homonukleaarsed sidemed). Paulingi skaalas on enamustele elementidele elektronegatiivsused teada. Kõvadus avaldub kui:

$\displaystyle\eta=\frac{1}{2}\frac{\partial{\,^2E}}{\partial{N\,^2}}\,_Z$

Kuna kõvaduse numbrilisi väärtuseid otseselt teada ei ole, siis teeme lihtsustuse, et nad sõltuvad samuti vastava elemendi elektronegatiivsusest.

$\displaystyle\Delta q = \frac{\chi_{\,1}-\chi_{\,2}}{\eta_{\,1}+\eta_{\,2}} \approx \frac{\chi_{\,1}-\chi_{\,2}}{\chi_{\,1}+\chi_{\,2}}$



In [ ]:
na_en=Na.electronegativity(scale='pauling')
cl_en=Cl.electronegativity(scale='pauling')

delta_q=(na_en-cl_en)/(na_en+cl_en)
print("Kloori osalaeng: %.2f, naatriumi osalaeng %.2f."%(delta_q, -delta_q))

Mulliken pakkus välja absoluutse elektronegatiivsuste skaala, kus elemendi elektronegatiivsus on tema ionisatsioonienergia ja elektronafiinsuse aritmeetiline keskmine:

$\displaystyle\chi =  \frac{\text{IE}+\text{EA}}{2}$

Kõvaduse valemit saab diferentseerida kolme määratud punkti abil -- üks elektron rohkem, üks elektron vähem ja tavaline valentsolek:

$\displaystyle\eta=\frac{1}{2}\frac{\partial{\,^2E}}{\partial{N\,^2}}\,_Z \approx \frac{E_{N+1}-2E_{N}+E_{N-1}}{2}$

Definitsiooni alusel on $E_{N-1}-E_N=\text{IE}$ ja $E_{N}-E_{N+1}=\text{EA}$, seega:

$\displaystyle\eta \approx \frac{\text{IE}-\text{EA}}{2}$

Rakendame Mullikeni elektronegatiivsustega taas laenguülekande valemit, seekord kasutades kõvadusi:

$\displaystyle\Delta q = \frac{\chi_{\,1}-\chi_{\,2}}{\eta_{\,1}+\eta_{\,2}}$

In [ ]:
na_ie=Na.ionenergies[1]
cl_ie=Cl.ionenergies[1]
na_ea=Na.electron_affinity
cl_ea=Cl.electron_affinity

na_chi=(na_ie+na_ea)/2
cl_chi=(cl_ie+cl_ea)/2
na_eta=(na_ie-na_ea)/2
cl_eta=(cl_ie-cl_ea)/2

delta_q=(na_chi-cl_chi)/(na_eta+cl_eta)
print("Kloori osalaeng: %.2f, naatriumi osalaeng %.2f."%(delta_q, -delta_q))

In [ ]:
correction=1.0
while correction < 0.01:
  correction=14.4*(delta_q**2)/2.82
  delta_q=(na_chi-cl_chi)/(na_eta+cl_eta+correction) 
  print(correction)
print(delta_q)

Nüüd siis DFT juurde. Arvutame NaCl ioonpaari vaakumis ning arvutame osalaengud. 

Osalaengute arvutamiseks on kolme tüüpi meetodeid:

 - Baasifunktsioonidel põhinevad laengujaotused -- lokaliseeritakse molekulaarorbitaalid aatomitele. Kvantkeemia või DFT arvutustest on teada orbitaalide asustused, seega saab lugeda kokku antud aatomile kuuluvate elektronide arvu ja lahutada maha tuumalaengu. Antud lähenemise põhiliseks puuduseks on see, et ühe aatomi elektrontihedus võib olla osaliselt kirjeldatud teise aatomi baasifunktsioonidega. See puudus on on tingitud arvutuskeemia fundamendaalsetest alustest: me ei tea kunagi "õiget" lainefunktsiooni.
 
 - Elektrostaatilise potentsiaali meetodid -- defineeritakse molekulist veidi eemal pind, millel arvutatakse erinevates punktides elektrostaatiline potentsiaal, mida elektrontihedus talle avaldab. Järgnevalt vaadeldakse olukorda, kus selle sama pinna sees on elektrontiheduse asemel punktlaengud nendes kohtades, kus olid varem tuumad. Üritatakse reprodutseerida elektrontiheduse elektrostaatilist potentsiaalijaotust antud pinnal, muutes punktlaengute väärtuseid. See lähenemine ei kirjelda hästi suurte molekulide keskmisi aatomeid ning võib anda erinevaid laenguid sõltuvalt konformatsioonist (nt metüülrühma vesinikud võivad saada veidi erineva osalaengu). 
 
 - Elektrontiheduse gradiendi analüüs -- elektrontiheduse väärtuse ja selle muutumise järgi jagatakse ruum aatomite vahel ära. Eeldatakse, et aatomituumade juures on elektrontihedus maksimaalne. Sinna, kus kahe tuuma vahel esineb elektrontiheduse miinimum, tõmmatakse piirid. See lähenemisviis on kõige paremini põhjendatud, kuid samuti ka kõige kulukam.
 
 
Arvutame NaCl Mullikeni ja CHELPG osalaengud. Esimene põhineb baasifunktsioonidel põhinev ning teine elektrostaatilise potentsiaali reprodutseerimisel.

In [ ]:
#Defineerime geomeetria
pair = Atoms('NaCl', positions=[(0, 0, 0),(0, 0, 2.82)])
write("NaCl.xyz", pair)
pair.set_pbc((False, False, False))
pair.center(vacuum=5)
HTML(atoms_to_html(pair))

Kuna Mullikeni ja CHELPG osalaenguid ASE-s ega GPAW-s defineeritud ei ole, siis kutsume välja teise arvutusprogrammi, Orca. Kasutame dzp baasi ja PBE funktsionaali ning seda geomeetriat, mille me ASE-ga just salvestasime.

In [ ]:
%%bash
. ~/.bashrc
/home/user/storage/software/orca_4/orca orca.inp > orca.out

Loeme saadud tulemusfaili orca.out

In [ ]:
f=open("orca.out").readlines()   #f on list, milles iga element on rida failist
for n in range(len(f)):          #Teeme tsükli ridade lugemiseks
  #Kui meid huvitavad osalaengud on antud real kirjas, siis ...
  if "CHELPG Charges" in f[n] or "MULLIKEN ATOMIC CHARGES" in f[n]: 
    #... prindime kuus rida alates leitud märksõnast
    for i in range(6):
      print(f[n+i])

Mõlemad osalaengud ennustavad küllaltki suurt laenguülekannet kloriidioonilt tagasi naatriumile.

Kuna naatrium ja kloor on mõlemad kolmanda perioodi elemendid ning me anname neile mõlemale sama suure baaslainefunktsioonide komplekti (dzp). Naatrium, olles lahutanud elektroni, on [Xe] elektronkonfiguratsiooniga samas kui kloor liites elektrooni savutab [Ar] elektronkonfiguratsiooni.

Tulemuseks on see, et tõenäolieslt kirjeldatakse osa kloori laengutihetust hoopis naatriumile kuuluvate baaslainefunktsioonidega. Sellest tulenevalt on saadud Mullikeni osalaeng nii madal. Seega on tõenäoliselt usaldusväärsem CHELPG osalaeng.

Katsetame ka arvutada Baderi osalaengut, mis põhineb elektrontiheduse gradiendi jaotamisel. Baderi analüüsi teostamiseks on vaja arvutusest väljastada elektrontiheduse fail (.cube formaat). Teeme seda GPAW-ga, kasutame analoogseid arvutusparameetreid kui enne: PBE funkstionaal ja dzp baas.

In [ ]:
#Define calculator and run simple PBE calculation
calc = GPAW(xc='PBE',
            mode='lcao',
            h=0.3,
            txt='NaCl-PBE.txt',
            basis='dzp')
pair.set_calculator(calc)
pair.get_potential_energy()
pair.calc.write('NaCl-PBE.gpw', mode='all')

#Peale arvutust väljastame ka elektrontiheduse
density = calc.get_all_electron_density(gridrefinement=4) * Bohr**3
#Salvestame tulemuse cube faili
write('NaCl-PBE.cube', pair, data=density)

Kutsume välja Baderi analüüsiprogrammi ja anname talle ette saadud faili.

In [ ]:
%%bash
. ~/.bashrc
/home/user/LOKT.02.048/week5/bader -p all_atom -p atom_index NaCl-PBE.cube

Prindime Baderi analüüsi tulemusfaili.

In [ ]:
f=open("ACF.dat").read()
print(f)

Arvutame elektronide arvust ioonide osalaengud

In [ ]:
f=open("ACF.dat").readlines()

Na_electrons=float(f[2].split()[4])
Cl_electrons=float(f[3].split()[4])

Na_protons=11
Cl_protons=17

print("Naatriumil on %.2f elektroni ja klooril %.2f elektroni."%(Na_electrons,Cl_electrons))
print("Naatriumi osalaeng on %.2f ja kloori osalaeng %.2f."%(Na_protons-Na_electrons, Cl_protons-Cl_electrons))

Visualiseerime järgnevalt leitud NaCl elektrontiheduse läbilõike kontuurgraafikuna, kasutades cube-faili, ning lisame joonisele Baderi leitud piiri.

In [ ]:
dens, atoms = read_cube_data('NaCl-PBE.cube')   
bader, atoms = read_cube_data('AtIndex.cube')     
x = len(dens) // 2 
dens = dens[x]
bader = bader[x]

x0, y0, z0 = atoms.positions[0]
y = np.linspace(0, atoms.cell[1, 1], len(dens), endpoint=False) - y0
z = np.linspace(0, atoms.cell[2, 2], len(dens[0]), endpoint=False) - z0
#print(y.shape, z.shape, dens.shape, bader.shape)
#print(atoms.positions)
#print(dens.min(), dens.mean(), dens.max())
plt.figure(figsize=(8, 8))


region=np.linspace(-3, 2, 8)
#region=np.linspace(-3, 2, 14)     # zoom in
region = [ 10**i for i in region]

contour=plt.contour(z, y, dens, region, norm=colors.PowerNorm(gamma=1./8.))

plt.contour(z, y, bader, [1.5], colors='k')
plt.axis(xmin=-2, xmax=6, ymin=-4, ymax=4)  
#plt.axis(xmin=-1, xmax=3, ymin=-2, ymax=2)  # zoom in
plt.clabel(contour, inline=1, rightside_up=1, fontsize=8)
plt.savefig('NaCl-bader.png')

Arvutasime ioonpaari vaakumis. Mis siis saab kui on tegu kristalliga? Kas te arvate, et kristallis on side ioonilisem või kovalentsem kui üksikus ioonpaaris?

In [ ]:
#Defineerime geomeetria
import ase.lattice
crystal = bulk('NaCl', 'rocksalt', a=5.64,cubic=True) 

HTML(atoms_to_html(crystal))

In [ ]:
#Define calculator and run simple PBE calculation
calc = GPAW(xc='PBE',
            mode='fd',
            h=0.3,
            kpts=(1,1,1),
            txt='crystal-PBE.txt',
            basis='dzp')
crystal.set_calculator(calc)
crystal.get_potential_energy()
crystal.calc.write('crystal-PBE.gpw', mode='all')

#Peale arvutust väljastame ka elektrontiheduse
density_cry = calc.get_all_electron_density(gridrefinement=4) * Bohr**3
#Salvestame tulemuse cube faili
write('crystal-PBE.cube', pair, data=density_cry)

Baderi analüüs

In [ ]:
%%bash
. ~/.bashrc
/home/user/LOKT.02.048/week5/bader -p all_atom -p atom_index crystal-PBE.cube

Prindime aatomilaengud

In [ ]:
f=open("ACF.dat").read()
print(f)

Tõenäoliselt perioodilisus tekitab Baderis vea. Kindlasti pole meil 800 elektroni ühes ühikrakus. Ja kindlasti pole klooril ja naatirumil võrdsed laengud. Prindime välja ruumala järgi arvutatud Baderi laengud.

In [ ]:
f=open("BCF.dat").read()
print(f)

Siin failis on näha kõigi leitud aatomite ruumiosade osalaengud. Osalaengud on küll ebareaalselt suured, kuid neid võib normaliseerida, arvestades, et summaarne elektronide arv peaks olema 28.

Antud väljundfailist on näha veel üks viga: aatomitel numbriga 1 on vahepeal CHARGE 72, vahepeal 127. Tõenäoliselt on Bader arvanud, et mõned kloorid on hoopis naatriumid ja vastupidi.

Arvestades, et kõik ~72 elektroniga aatomid on naatriumid ja ~127 elektroniga laengud on kloorid, võime summeerida ja normeerida laengud ning väljastada keskmise naatirumi ja kloori osalaengu.

In [ ]:
f=open("BCF.dat").readlines()
Na_electrons=[]
Cl_electrons=[]
total_electrons=28

for i in f[2:-1]:
  if float(i.split()[4])<100.:
    Na_electrons.append(float(i.split()[4]))
  elif float(i.split()[4])>100.:
    Cl_electrons.append(float(i.split()[4]))

norm_factor=28/(sum(Na_electrons)+sum(Cl_electrons))
Na_electrons_norm=(norm_factor*sum(Na_electrons))
Cl_electrons_norm=(norm_factor*sum(Cl_electrons))

Na_protons=11
Cl_protons=17

print("Naatriumil on %.2f elektroni ja klooril %.2f elektroni."%(Na_electrons_norm,Cl_electrons_norm))
print("Naatriumi osalaeng on %.2f ja kloori osalaeng %.2f."%(Na_protons-Na_electrons_norm, Cl_protons-Cl_electrons_norm))

Visualiseerime ühikraku laengutiheduse.

In [ ]:
dens, atoms = read_cube_data('crystal-PBE.cube')   
bader, atoms = read_cube_data('AtIndex.cube')     
x = len(dens) // 2 
dens = dens[x]
bader = bader[x]

x0, y0, z0 = atoms.positions[0]
y = np.linspace(0, atoms.cell[1, 1], len(dens), endpoint=False) - y0
z = np.linspace(0, atoms.cell[2, 2], len(dens[0]), endpoint=False) - z0
#print(y.shape, z.shape, dens.shape, bader.shape)
#print(atoms.positions)
#print(dens.min(), dens.mean(), dens.max())
plt.figure(figsize=(8, 8))


region=np.linspace(-3, 2, 8)
#region=np.linspace(-3, 2, 14)     # zoom in
region = [ 10**i for i in region]

contour=plt.contour(z, y, dens, region, norm=colors.PowerNorm(gamma=1./8.))

plt.contour(z, y, bader, [1.5], colors='k')
plt.axis(xmin=-5.64, xmax=8.64, ymin=-5.64, ymax=5.64)  
#plt.axis(xmin=-1, xmax=3, ymin=-2, ymax=2)  # zoom in
plt.clabel(contour, inline=1, rightside_up=1, fontsize=8)
plt.savefig('crystal-bader.png')